In [ ]:
#Etapa 1: Entender el primer dataset (Paradas de trenes, tranvías, metro, funiculares y ferrocariles)

import pandas as pd

file_path = 'transporte_publico_bcn.csv'
data = pd.read_csv(file_path)

data.head(), data.columns

In [ ]:
#Etapa 2: Miramos los datos más en su globalidad y si hay algún nul en las columnas que nos interesan

print(data.info())
print(data.isnull().sum())

In [ ]:
#Etapa 3: Visualizaremos el dataset en un mapa con filtros dinámicos para entenderlo mejor

import folium
from folium import FeatureGroup
from folium.plugins import FeatureGroupSubGroup
import pandas as pd

file_path = 'transporte_publico_bcn.csv'
output_path = '0_mapa_transporte_publico_bcn_except_bus.html'

data = pd.read_csv(file_path)

color_list = ['#FF5733', '#33FF57', '#3357FF', '#F33FFF', '#FFD700', '#FF33F5', '#F5FF33', '#FF8F33', '#8FFF33', '#A52A2A']
# Colores específicos para Funicular y RENFE
specific_colors = {
    'Funicular': '#8B4513',  # Marrón oscuro para Funicular
    'RENFE': '#000080'       # Azul marino para RENFE
}


mapa = folium.Map(location=[41.3851, 2.1734], zoom_start=12)

# Diccionario para almacenar colores por cada `NOM_CAPA`
color_dict = {}

# Crear grupos de capas
group_all = folium.FeatureGroup(name='All Transport Types')
group_dict = {}

for i, nom_capa in enumerate(data['NOM_CAPA'].unique()):
    group_dict[nom_capa] = FeatureGroupSubGroup(group_all, nom_capa)
    color_dict[nom_capa] = specific_colors.get(nom_capa, color_list[i % len(color_list)])

# Añadir marcadores a los grupos de capas
for nom_capa, group in group_dict.items():
    for _, row in data[data['NOM_CAPA'] == nom_capa].iterrows():
        if 'central' not in row['NOM_CAPA'].lower():  # Filtrar el punto central
            folium.CircleMarker(
                location=[row['LATITUD'], row['LONGITUD']],
                radius=8,
                color=color_dict[nom_capa],
                fill=True,
                fill_color=color_dict[nom_capa],
                fill_opacity=0.7,
                popup=row['EQUIPAMENT']
            ).add_to(group)

# Añadir los grupos de capas al mapa
for group in group_dict.values():
    group.add_to(mapa)

group_all.add_to(mapa)

# Añadir control de capas
folium.LayerControl(collapsed=False).add_to(mapa)

# Añadir leyenda
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: auto; 
            background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
    &nbsp;<b>Transport Types</b><br>
'''
for nom_capa, color in color_dict.items():
    legend_html += f'&nbsp;<i style="background:{color};width:10px;height:10px;display:inline-block;"></i>&nbsp;{nom_capa}<br>'

legend_html += '</div>'
mapa.get_root().html.add_child(folium.Element(legend_html))


mapa.save(output_path)

mapa

In [ ]:
#Etapa 4: Entender el segundo dataset (paradas de autobuses en Barcelona)

import pandas as pd

file_path = 'ESTACIONS_BUS.csv'
data = pd.read_csv(file_path)

data.head(), data.columns

In [ ]:
#Etapa 5: Miramos los datos más en su globalidad y si hay algún nul en las columnas que nos interesan

print(data.info())
print(data.isnull().sum())

In [ ]:
#Etapa 6: Visualizaremos el dataset en un mapa con filtros dinámicos para entenderlo mejor - Separación bus i nit bus

import folium
from folium.plugins import MarkerCluster
import pandas as pd


file_path = 'ESTACIONS_BUS.csv'


data = pd.read_csv(file_path)
data.head()

mapa = folium.Map(location=[41.3851, 2.1734], zoom_start=12)

# Crear grupos de capas
grupo_nitbus = folium.FeatureGroup(name='NITBUS')
grupo_bus = folium.FeatureGroup(name='BUS')

# Añadir un clúster de marcadores para agrupar los puntos
marker_cluster_nitbus = MarkerCluster().add_to(grupo_nitbus)
marker_cluster_bus = MarkerCluster().add_to(grupo_bus)

# Añadir marcadores a los clústeres correspondientes
for _, row in data.iterrows():
    if 'NITBUS' in row['EQUIPAMENT']:
        folium.Marker(
            location=[row['LATITUD'], row['LONGITUD']],
            popup=row['EQUIPAMENT']
        ).add_to(marker_cluster_nitbus)
    else:
        folium.Marker(
            location=[row['LATITUD'], row['LONGITUD']],
            popup=row['EQUIPAMENT']
        ).add_to(marker_cluster_bus)

# Añadir los grupos de capas al mapa
grupo_nitbus.add_to(mapa)
grupo_bus.add_to(mapa)

# Añadir control de capas
folium.LayerControl(collapsed=False).add_to(mapa)


output_path = '1_mapa_bus_bcn.html'
mapa.save(output_path)

mapa

In [ ]:
#Etapa 7: realizar la fusión de los datasets y el analísis con los puntos de referencia y un radio de 0,75km alrededor - export:archivo csv de scoring + mapa:

import pandas as pd
import folium
from folium.plugins import HeatMap
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import distance


bus_stops = pd.read_csv('ESTACIONS_BUS.csv')
train_stops = pd.read_csv('transporte_publico_bcn.csv')
points = gpd.read_file('hundredpoints.geojson')

bus_stops = bus_stops.rename(columns={'LATITUD': 'latitud', 'LONGITUD': 'longitud'})
train_stops = train_stops.rename(columns={'LATITUD': 'latitud', 'LONGITUD': 'longitud'})
points = points.rename(columns={'lat': 'latitud', 'long': 'longitud'})

bus_stops['tipo'] = 'Bus'
train_stops['tipo'] = 'Tren'

stops = pd.concat([bus_stops, train_stops], ignore_index=True)

def count_nearby_stops(point, stops, radius_km=0.75):
    center = (point.y, point.x)
    count = 0
    for _, stop in stops.iterrows():
        stop_point = (stop['latitud'], stop['longitud'])
        if distance(center, stop_point).km <= radius_km:
            count += 1
    return count

points['puntuacion'] = points.geometry.apply(lambda point: count_nearby_stops(point, stops))

# Normalizar las puntuaciones para que estén entre 0 y 15
def normalizar_puntuaciones(puntuaciones, min_val=0, max_val=15):
    min_puntuacion = puntuaciones.min()
    max_puntuacion = puntuaciones.max()
    
    # Evitar división por cero si todas las puntuaciones son iguales
    if min_puntuacion == max_puntuacion:
        return [min_val for _ in puntuaciones]
    
    puntuaciones_normalizadas = [
        ((puntuacion - min_puntuacion) / (max_puntuacion - min_puntuacion)) * (max_val - min_val) + min_val
        for puntuacion in puntuaciones
    ]
    
    return puntuaciones_normalizadas

# Normalizar las puntuaciones
points['scoring'] = normalizar_puntuaciones(points['puntuacion'])


points = points[['geometry', 'scoring']]

output_csv_path = 'puntos_analisis_con_scoring_transporte_público.csv'
points.to_csv(output_csv_path, index=True)


# Crear el mapa 
map_center = [points.geometry.y.mean(), points.geometry.x.mean()]
mapa = folium.Map(location=map_center, zoom_start=12)


heat_data = [[point.y, point.x, row.scoring] for point, row in zip(points.geometry, points.itertuples())]

HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(mapa)

for _, row in points.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.2,
        popup=f"Puntuación: {row['scoring']:.2f}"
    ).add_to(mapa)


output_path = '2_mapa_puntuación_análisis_final.html'
mapa.save(output_path)

mapa

In [ ]:
#Etapa 8 : Analizar los resultados obtenidos (puntuaciones)

import pandas as pd
import plotly.express as px
import geopandas as gpd
from shapely import wkt


file_path = 'puntos_analisis_con_scoring_transporte_público.csv'

data = pd.read_csv(file_path)

print(data.head())

estadisticas_descriptivas = data['scoring'].describe()
print("Estadísticas Descriptivas:")
print(estadisticas_descriptivas)

# Histograma 
fig_hist = px.histogram(data, x='scornig', nbins=15, title='Histograma de Puntuaciones Normalizadas')
fig_hist.show()

# Boxplot 
fig_box = px.box(data, x='scoring', title='Boxplot de Puntuaciones Normalizadas')
fig_box.show()

# Convertir la columna 'geometry' a objetos geométricos
data['geometry'] = data['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(data, geometry='geometry')

# Mapa
gdf['lon'] = gdf.geometry.x
gdf['lat'] = gdf.geometry.y
fig_map = px.scatter_mapbox(gdf, lat="lat", lon="lon", color="scoring", 
                            color_continuous_scale="Viridis", size_max=15, zoom=10,
                            mapbox_style="carto-positron", title='Mapa de Puntos GPS con Puntuación Normalizada')
fig_map.show()
